### Basic global settings trials including ncluster, method and data used

In [1]:
import os
import json
import itertools
import pandas as pd
from functions import *
%set_env GUROBI_HOME=/usr/local/gurobi/linux64

env: GUROBI_HOME=/usr/local/gurobi/linux64


In [2]:
with open('config.json', 'r') as f:
    config_all = json.loads(f.read())

config = config_all['Texas_7_years_1']
inputs = config['inputs']
settings = config['settings']
print(inputs)
pv_all = np.genfromtxt(
    os.path.join(config['root'], config['pv_path']), delimiter=',', filling_values=0)
wind_all = np.genfromtxt(
    os.path.join(config['root'], config['wind_path']), delimiter=',', filling_values=0)
load_all = np.genfromtxt(
    os.path.join(config['root'], config['load_path']), delimiter=',', filling_values=0)  # Data load

{'c_bat': 200000, 'c_bat_power': 70000, 'c_pv': 1000000, 'c_wind': 1500000, 'c_gen_inv': 1000000, 'c_gen_a': 0.1, 'c_gen_b': 30, 'up_time': 6, 'down_time': 6, 'energy_cap': 200, 'power_cap': 200, 'gen_cap': 0, 'mdc': 50, 'effi': 0.9, 'rps': 0.8, 'note_2': 'This property is set to indicate what are above properties for model. Change with model parameters.', 'solver': 1, 'print_log': 0, 'timelimit': 210240, 'note_3': 'This property is set to indicate what are above properties for solver. No need to change.'}


In [3]:
day_num = settings['day_num']
time_set = np.arange(24*day_num)  # Time horizon
if 'profile_id' in settings:
    profile_id = settings['profile_id']
    renewable = [wind_all[time_set, profile_id],pv_all[time_set, profile_id]]
else:
    # Denote that first profile is used and no other profile exists.
    profile_id = -1
    renewable = [wind_all[time_set], pv_all[time_set]]
load = load_all[time_set]*100  # Data extract

In [4]:
list_temp = itertools.product([100000,150000],[50000,30000],[500000,750000],[1000000,1250000])
backups = pd.DataFrame(columns=['c_bat','c_bat_power','c_pv','c_wind'],data=list_temp)
for backup in backups.index:
    print("Index :" + str(backup))
    update = backups.loc[backup,:].to_dict()
    for item in ['c_bat','c_bat_power','c_pv','c_wind']:
        inputs[item] = update[item]
    config['inputs'] = inputs
    expected = pd.read_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-{}/expected/expected_2555_{}_1e-4.csv'.format(profile_id,backup), index_col=0).loc[0,:].to_dict()
    sim_features_df = pd.read_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-{}/features/features_2555_{}_1e-4.csv'.format(profile_id,backup), index_col=0)
    df = run_trials(config, wind_all, pv_all, load_all, expected, sim_features_df[['renewable_cap_0','renewable_cap_1','max_energy','max_power']])
    df.set_index('description').to_csv(
        config['save_root']+settings['data']+('_Int_' if inputs['gen_cap'] else '_')+('Renewable'+str(profile_id)+'_' if 'profile_id' in settings else '_')+str(settings['day_num'])+'Days_'+str(settings['period'])+'HrsPerPeriod_{}.csv'.format(backup))
print("All Done!")

Index :0
1th trial:  2.5627708435058594
2th trial:  2.249434232711792
3th trial:  2.224001169204712
4th trial:  2.3237569332122803
5th trial:  2.084604263305664
6th trial:  1.6639928817749023
7th trial:  1.9650938510894775
8th trial:  1.851499319076538
9th trial:  1.895812749862671
10th trial:  1.8389956951141357
11th trial:  2.889582395553589
12th trial:  2.1869733333587646
13th trial:  2.1690542697906494
14th trial:  2.0661866664886475
15th trial:  2.159688711166382
16th trial:  4.304081678390503
17th trial:  3.3032748699188232
18th trial:  3.43255877494812
19th trial:  3.4311323165893555
20th trial:  3.4151771068573
21th trial:  2.9718897342681885
22th trial:  3.2701873779296875
23th trial:  3.2143442630767822
24th trial:  3.126596689224243
25th trial:  3.3851335048675537
26th trial:  4.266472816467285
27th trial:  3.711275100708008
28th trial:  3.5991644859313965
29th trial:  3.4563848972320557
30th trial:  3.478930711746216
31th trial:  5.253399848937988
32th trial:  4.98844814300

89th trial:  10.940982818603516
90th trial:  11.21079707145691
Index :3
1th trial:  2.4213035106658936
2th trial:  2.401855230331421
3th trial:  2.057738780975342
4th trial:  2.1089720726013184
5th trial:  2.2367818355560303
6th trial:  1.6831414699554443
7th trial:  1.9587864875793457
8th trial:  1.8228390216827393
9th trial:  1.9183874130249023
10th trial:  1.8647243976593018
11th trial:  2.7627387046813965
12th trial:  2.291076183319092
13th trial:  2.1975483894348145
14th trial:  2.1898515224456787
15th trial:  2.3038601875305176
16th trial:  3.4645540714263916
17th trial:  3.414320707321167
18th trial:  3.614022970199585
19th trial:  3.595106363296509
20th trial:  3.7559356689453125
21th trial:  3.2873799800872803
22th trial:  3.4619243144989014
23th trial:  3.3397812843322754
24th trial:  3.3828415870666504
25th trial:  3.729234457015991
26th trial:  3.9348816871643066
27th trial:  3.6398777961730957
28th trial:  3.6834239959716797
29th trial:  3.316405773162842
30th trial:  3.40

81th trial:  11.83774209022522
82th trial:  11.050050973892212
83th trial:  11.72983694076538
84th trial:  10.616137981414795
85th trial:  11.589905738830566
86th trial:  14.162509679794312
87th trial:  12.509711027145386
88th trial:  11.470911979675293
89th trial:  11.732462406158447
90th trial:  11.423117637634277
Index :6
1th trial:  1.9019782543182373
2th trial:  2.556185007095337
3th trial:  2.1066441535949707
4th trial:  2.1260013580322266
5th trial:  2.07742977142334
6th trial:  1.638174295425415
7th trial:  1.9173521995544434
8th trial:  1.8999056816101074
9th trial:  1.9680452346801758
10th trial:  1.8598294258117676
11th trial:  2.437971353530884
12th trial:  2.1715056896209717
13th trial:  1.9231326580047607
14th trial:  2.1209194660186768
15th trial:  2.395134687423706
16th trial:  4.1870434284210205
17th trial:  3.381741523742676
18th trial:  3.4605789184570312
19th trial:  3.923707962036133
20th trial:  4.041750431060791
21th trial:  2.997523784637451
22th trial:  3.34075

73th trial:  10.064560890197754
74th trial:  9.018868923187256
75th trial:  9.21839690208435
76th trial:  13.973690509796143
77th trial:  10.692196369171143
78th trial:  11.157489776611328
79th trial:  11.844458818435669
80th trial:  12.627874374389648
81th trial:  12.461703300476074
82th trial:  10.881085395812988
83th trial:  11.87017273902893
84th trial:  11.646166801452637
85th trial:  10.759777545928955
86th trial:  12.122761487960815
87th trial:  13.347851037979126
88th trial:  13.423697471618652
89th trial:  13.89930534362793
90th trial:  12.614790439605713
Index :9
1th trial:  4.007616758346558
2th trial:  2.5845327377319336
3th trial:  2.5507500171661377
4th trial:  2.6851987838745117
5th trial:  2.2483069896698
6th trial:  1.7289328575134277
7th trial:  1.956993579864502
8th trial:  1.7805476188659668
9th trial:  1.9849472045898438
10th trial:  2.0485799312591553
11th trial:  2.5398361682891846
12th trial:  2.2170000076293945
13th trial:  2.2855169773101807
14th trial:  2.421

65th trial:  10.187452793121338
66th trial:  10.60847020149231
67th trial:  10.981478214263916
68th trial:  10.583680391311646
69th trial:  11.179323673248291
70th trial:  13.574417114257812
71th trial:  15.88329005241394
72th trial:  9.9775390625
73th trial:  9.673758506774902
74th trial:  11.275388240814209
75th trial:  11.233637571334839
76th trial:  21.037285566329956
77th trial:  14.709937572479248
78th trial:  13.659750699996948
79th trial:  11.790457725524902
80th trial:  12.046840190887451
81th trial:  14.390284061431885
82th trial:  14.200743913650513
83th trial:  13.444083213806152
84th trial:  15.686525583267212
85th trial:  14.80711841583252
86th trial:  20.663108825683594
87th trial:  11.300110101699829
88th trial:  11.096768379211426
89th trial:  13.426039218902588
90th trial:  15.965340614318848
Index :12
1th trial:  4.8552470207214355
2th trial:  4.208538293838501
3th trial:  3.078604221343994
4th trial:  3.2875795364379883
5th trial:  4.3618690967559814
6th trial:  3.2

57th trial:  6.897901296615601
58th trial:  7.030189275741577
59th trial:  6.755683183670044
60th trial:  6.721349239349365
61th trial:  13.095110416412354
62th trial:  12.139597415924072
63th trial:  10.996066093444824
64th trial:  10.322468757629395
65th trial:  10.865887641906738
66th trial:  12.6946382522583
67th trial:  11.719543218612671
68th trial:  11.345196008682251
69th trial:  10.28075647354126
70th trial:  10.801334381103516
71th trial:  15.767390727996826
72th trial:  12.642583131790161
73th trial:  12.890673160552979
74th trial:  13.098325729370117
75th trial:  14.496981143951416
76th trial:  18.507264137268066
77th trial:  11.02179217338562
78th trial:  11.060664653778076
79th trial:  11.17264699935913
80th trial:  11.281931400299072
81th trial:  12.741358757019043
82th trial:  13.185085535049438
83th trial:  14.650627136230469
84th trial:  16.0322744846344
85th trial:  14.120338439941406
86th trial:  21.646695613861084
87th trial:  13.390663146972656
88th trial:  14.694